## RAG란?
- 자연어 처리(NLP) 분야의 혁신적인 기술로, 기존 모델의 한계를 넘어서 정보 검색과 생성을 통합하는 방법론
- 풍부한 정보를 담고 있는 대규모 문서 데이터베이스에서 관련정보를 검색하고, 
- 이를 통해 언어 모델이 더 정확하고 상세한 답변을 생성 할 수 있습니다.

### 핵심 개념
- **검색(Retrieval)**: 질문과 관련된 문서나 정보를 벡터 유사도 기반으로 찾기
- **증강(Augmented)**: 찾은 정보를 언어 모델의 컨텍스트에 추가
- **생성(Generation)**: 증강된 정보를 바탕으로 더 정확하고 신뢰할 수 있는 답변 생성


### RAG의 장점
1. 최신 정보 반영 가능 (지식 갱신이 간편)
2. 출처 기반 응답 제공
3. 도메인 특화 응답 가능

### RAG의 기본구조

![RAG.png](./images/RAG.png)



**RAG-FLOW**
![RAG_Flow.png](./images/RAG_Flow.png)


## PDF를 문서로 사용해서 RAG 시스템 구축하기
- 다음 실습 예시는 기본적인 RAG 구조를 구현합니다.
- 추후, 개발을 진행할때는 각각의 구현 상황에 맞는 모듈로 바꾸는 것 만으로도 여러분들만의 RAG 시스템을 구축 할 수 있습니다.

- 예시  
지금 예시에서는 PDF를 읽어와 문서를 저장하고 있지만, 제공되는 문서가 Text일 경우 Docuemnt Loader 모듈을 바꾸어 사용할 수 있습니다.  

**실습자료**
- 제목 : AI.GOV 해외동향 2025-1호
- 링크 : [한국지능사회정보진흥원](https://www.innovation.go.kr/ucms/bbs/B0000051/view.do?nttId=17774&menuNo=300145&pageIndex=)  
- 출처 : NIA 한국지증정보원원

*링크에서 파일을 다운로드 받은후 최상단 data 폴더에 위치시켜주세요*

### Step 1 : 문서 로드(Document Load)

In [1]:
# langchain_community : Langchain 핵심 기능 외에 커뮤니티가 기여하고 유지보수하는 외부 서비스 패키지지
%pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
# python으로 PDF문서를 빠르고 쉽게 열고 내용을 추출하거나 파일을 수정 및 이미지 저장을 해주는 라이브러리
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [3]:
# 1. DoucmentLoader
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("data/[AI.GOV_해외동향]_2025-1호.pdf")

docs = loader.load()

print(f"1번째 문서 내용 : {docs[0].page_content}")
print(f"총 문서의 갯수 : {len(docs)}")

1번째 문서 내용 : 1
해외 에이전트 기술 및 서비스 개발 동향AI 
을 보완하는 기술 현황LLM RAG 2.0 
정부의 적극적 활용 정책과 반향 미국 영국 사례AI : ·
주요국 인프라 투자 경쟁 가속화AI 
최신 보고서 의 미래 예견적 거버넌스 전략[OECD ] AI :
총 문서의 갯수 : 19


메타데이터 확인해보기

In [4]:
print(docs[0].__dict__)

{'id': None, 'metadata': {'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-03-25T14:20:25+09:00', 'author': '', 'moddate': '2025-03-25T14:20:25+09:00', 'title': '250324_AI.GOV txÙ¥ 2025-1(\\–).hwpx', 'source': 'data/[AI.GOV_해외동향]_2025-1호.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, 'page_content': '1\n해외 에이전트 기술 및 서비스 개발 동향AI \n을 보완하는 기술 현황LLM RAG 2.0 \n정부의 적극적 활용 정책과 반향 미국 영국 사례AI : ·\n주요국 인프라 투자 경쟁 가속화AI \n최신 보고서 의 미래 예견적 거버넌스 전략[OECD ] AI :', 'type': 'Document'}


### Step 2 : 문서 분할하기 (Text Splitter)

### Character Text Splitter
`CharacterTextSplitter`는 가장 기본적인 텍스트 분할 도구입니다.  
특정 문자(separator)를 기준으로 텍스트를 나누어 작은 청크(chunk)로 만들어줍니다.

**chunk란?**  
문장을 분석/처리 하기 쉽게 텍스트를 작은 단위로 나눈 조각을 의미합니다.

기본 사용법

CharacterTextSplitter의 기본 구조는 다음과 같습니다:

**주요 매개변수**
- `separator`: 텍스트를 나눌 기준 문자 (기본값: `"\n\n"`)
- `chunk_size`: 각 청크의 최대 문자 수
- `chunk_overlap`: 인접한 청크 간 겹치는 문자 수

Chunk Overlap이해하기

In [5]:
# Chunck Overlap 이해하기
from langchain_text_splitters import CharacterTextSplitter

sample_text = "이것은 오버랩 개념을 설명하기 위한 예시 문장입니다. 문맥이 끊어지지 않도록 도와줍니다."

# 1. Overlap 없이 분할하는 Splitter 생성
no_overlap_splitter = CharacterTextSplitter(
    separator=" ",         # 띄어쓰기 단위로 분할
    chunk_size=10,         # Chunk 최대 크기를 25자로 설정
    chunk_overlap=5,       # 겹치는 부분 없음
)

# 2. 텍스트 분할 실행
chunks = no_overlap_splitter.split_text(sample_text)

In [6]:
print("--- Overlap이 없을 때 (chunk_overlap=0) ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: \"{chunk}\"")


--- Overlap이 없을 때 (chunk_overlap=0) ---
Chunk 1: "이것은 오버랩"
Chunk 2: "오버랩 개념을"
Chunk 3: "개념을 설명하기"
Chunk 4: "설명하기 위한 예시"
Chunk 5: "예시 문장입니다."
Chunk 6: "문맥이 끊어지지"
Chunk 7: "끊어지지 않도록"
Chunk 8: "않도록 도와줍니다."


In [7]:
print("--- Overlap이 있을 때 (chunk_overlap=10) ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: \"{chunk}\"")

--- Overlap이 있을 때 (chunk_overlap=10) ---
Chunk 1: "이것은 오버랩"
Chunk 2: "오버랩 개념을"
Chunk 3: "개념을 설명하기"
Chunk 4: "설명하기 위한 예시"
Chunk 5: "예시 문장입니다."
Chunk 6: "문맥이 끊어지지"
Chunk 7: "끊어지지 않도록"
Chunk 8: "않도록 도와줍니다."


띄워쓰기가 없는 아주 긴 단어라면?

In [8]:
long_word_text = "이것은테스트입니다슈퍼울트라하이퍼메가캡숑단어"

# chunk_size를 10으로 아주 작게 설정
long_word_splitter = CharacterTextSplitter(
    # separator=" ",         # 띄어쓰기 단위로 분할
    chunk_size=10,         # Chunk 최대 크기를 10자로 설정
    chunk_overlap=0,       # 겹치는 부분 없음
)

chunks = long_word_splitter.split_text(long_word_text)

print(f"--- chunk_size=10 설정 결과 ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} (길이: {len(chunk)}): \"{chunk}\"")

--- chunk_size=10 설정 결과 ---
Chunk 1 (길이: 23): "이것은테스트입니다슈퍼울트라하이퍼메가캡숑단어"


### 작동 우선순위 (Hierarchy)
- 코드는 설정된 separators 리스트의 왼쪽부터 순서대로 적용하여 자를 위치를 찾습니다.

```Python
separators=["\n\n", "\n", ". ", " "]
```
1순위: 문단 (\n\n)
- 가장 먼저 전체 문단을 통째로 담으려 시도합니다.  
(설정된 크기를 넘지 않으면 문단 전체가 하나의 청크가 됨)  
  
2순위: 줄바꿈 (\n)  
- 문단이 너무 길면, 줄바꿈 단위로 쪼갭니다.  
  
3순위: 문장 (. )  
- 그래도 길면, 마침표를 기준으로 문장 단위로 쪼갭니다.
  
4순위: 단어/공백 ( )  
- 문장마저 너무 길면, 띄어쓰기 단위로 쪼갭니다.

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # 바꿔써보기

# 2. 문서 분할 
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " "], # 기본값
    chunk_size=200, # 최대 크기를 설정
    chunk_overlap=20 
)

test_text = """인공지능 기술이 발전함에 따라 거대언어모델(LLM)의 활용도가 높아지고 있습니다. LLM은 방대한 데이터를 학습하여 자연스러운 텍스트를 생성하는 데 탁월한 능력을 보입니다.

하지만 모델이 학습하지 않은 최신 정보나 기업 내부의 비공개 데이터에 대해서는 답변하지 못하거나, 사실이 아닌 내용을 사실인 것처럼 말하는 환각(Hallucination) 현상이 발생할 수 있습니다. 이러한 한계를 극복하기 위해 등장한 기술이 바로 검색 증강 생성(RAG)입니다.

RAG는 사용자의 질문이 들어오면 먼저 외부 데이터베이스에서 관련 문서를 검색합니다. 그 후 검색된 정확한 정보를 프롬프트에 포함하여 LLM이 답변을 생성하도록 유도합니다. 이 과정을 통해 AI는 더욱 신뢰성 있고 구체적인 답변을 제공할 수 있게 됩니다."""

chunks = recursive_splitter.split_text(test_text)

print(f"--- chunk_size=10 설정 결과 ---")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1} (길이: {len(chunk)}): \"{chunk}\"")

--- chunk_size=10 설정 결과 ---
Chunk 1 (길이: 96): "인공지능 기술이 발전함에 따라 거대언어모델(LLM)의 활용도가 높아지고 있습니다. LLM은 방대한 데이터를 학습하여 자연스러운 텍스트를 생성하는 데 탁월한 능력을 보입니다."
Chunk 2 (길이: 156): "하지만 모델이 학습하지 않은 최신 정보나 기업 내부의 비공개 데이터에 대해서는 답변하지 못하거나, 사실이 아닌 내용을 사실인 것처럼 말하는 환각(Hallucination) 현상이 발생할 수 있습니다. 이러한 한계를 극복하기 위해 등장한 기술이 바로 검색 증강 생성(RAG)입니다."
Chunk 3 (길이: 142): "RAG는 사용자의 질문이 들어오면 먼저 외부 데이터베이스에서 관련 문서를 검색합니다. 그 후 검색된 정확한 정보를 프롬프트에 포함하여 LLM이 답변을 생성하도록 유도합니다. 이 과정을 통해 AI는 더욱 신뢰성 있고 구체적인 답변을 제공할 수 있게 됩니다."


다시 RAG 구성으로

In [10]:
# 2. TextSplitter
from langchain_text_splitters import CharacterTextSplitter

# 기본 설정으로 텍스트 분할기 생성
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=200,     # 각 청크 최대 200자
    chunk_overlap=20   # 20자씩 겹침
)

split_documents = text_splitter.split_documents(docs)

print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 131


분할결과

In [11]:
print(split_documents[12].page_content)

해외동향 호AI.GOV 2025-1 5
에이전트의 유형AI Ⅱ
 ▸업무용 에이전트의 유형은 챗봇 컴퓨터 사용 에이전트AI AI , (CUA: Computer-Using 
다중 에이전트 시스템 등으로 구분될 수 있Agent), 음
   - 챗봇 검색 증강 생성 기술을 활용한 외부 지식 정보 접근 외부 시스템과 AI : RAG( ) , 상호


### Step 3 : 임베딩 준비하기 (Embedding)

#### Embedding이란?
**텍스트 → 수치 벡터(vector)** 로 변환하는 작업  
즉, 사람의 언어를 컴퓨터가 이해할 수 있는 형식으로 바꾸는 것.

`RAG`에서 관련있는 문서를 검색 해 올 수 있어야 하기 때문에, 문서를 벡터화 시켜, 의미가 비슷한 문서를 찾을 수 있다.

#### OpenAIEmbeddings
- OpenAIEmbeddings는 LangChain에서 OpenAI의 강력한 임베딩 모델을 사용하기 위한 도구입니다.  


**모델별 비교표**

| 모델명 | 비용 (1백만 토큰 당) | 기본 벡터 차원 | 특징 및 장단점 |
| :--- | :--- | :--- | :--- |
| **`text-embedding-3-small`** | **$0.02** | 1536 | **(추천)** **가성비가 가장 뛰어난 최신 모델.** `ada-002`보다 훨씬 저렴하지만 성능(특히 다국어)은 더 우수함. 대부분의 경우에 추천됨. |
| **`text-embedding-3-large`** | $0.13 | 3072 | **최고 성능 모델.** 가장 높은 정확도를 제공하며, 미묘하고 복잡한 의미를 파악하는 데 가장 강력함. 비용과 벡터 저장 공간이 더 필요함. |
| `text-embedding-ada-002` | $0.10 | 1536 | **(구세대 모델)** 과거에 가장 널리 쓰였던 모델. 특별한 이유가 없다면 이제는 `3-small` 모델 사용이 모든 면에서 유리함. |


In [12]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# OpenAI API 환경변수 값 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"OPENAI_API_KEY가 설정되어 있나요?: {openai_api_key[:10]}...")

OPENAI_API_KEY가 설정되어 있나요?: sk-proj-rr...


In [13]:
%pip install langchain_openai


Note: you may need to restart the kernel to use updated packages.


In [14]:
# 3. Embedding
from langchain_openai import OpenAIEmbeddings

# 1. OpenAI 임베딩 모델 초기화 (ChatGPT-4o-mini와 호환)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # OpenAI의 최신 임베딩 모델
    # API 키는 환경변수 OPENAI_API_KEY에서 자동으로 읽어옵니다
)

print(f"모델명: {embeddings.model}")

모델명: text-embedding-3-small


임베딩 확인해보기

In [15]:
document_vectors = embeddings.embed_documents(chunks)

print(f"--- 문서 임베딩 결과 ---")
print(f"총 {len(document_vectors)}개의 Chunk가 벡터로 변환되었습니다.")
print(f"각 벡터의 차원(길이): {len(document_vectors[0])}")
print(f"첫 번째 Chunk의 벡터(일부): {document_vectors[0]}...")

--- 문서 임베딩 결과 ---
총 3개의 Chunk가 벡터로 변환되었습니다.
각 벡터의 차원(길이): 1536
첫 번째 Chunk의 벡터(일부): [0.027627112343907356, 0.05728862062096596, 0.02551133930683136, 0.0032957231160253286, 0.013162548653781414, -0.018869033083319664, -0.021259449422359467, 0.03287585452198982, -0.011138323694467545, 0.04434985667467117, -0.01959124393761158, -0.01913350448012352, -0.049802038818597794, -0.0007374689448624849, 0.005380980204790831, -0.005411495920270681, -0.003982332069426775, -0.020730506628751755, 0.017526330426335335, 0.029783573001623154, -0.011443483643233776, 0.009200560860335827, 0.011514686979353428, -0.02695576101541519, 0.018563872203230858, -0.0012765841092914343, 0.01387458760291338, 0.03826700896024704, 0.01907247304916382, -0.013081173412501812, 0.012491198256611824, -0.017943382263183594, 0.00931753870099783, 0.004956299904733896, 0.0016389611409977078, 0.03950798884034157, 0.02555202692747116, 0.017475470900535583, -0.016661711037158966, -0.0007069529965519905, -0.0027388071175664663, -0.

### Step 4 : Vector Store 생성 및 문서 임베딩

In [16]:
%pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [25]:
# 4. VectorStore
from langchain_community.vectorstores import FAISS
#벡터 데이터 베이스를 만들기 위해 (split한 문서들과 embedding에 사용할 모델을 인자로 넣어둔다.)
db = FAISS.from_documents(documents=split_documents, embedding=embeddings)

검색해보기

In [26]:
query = "주종별 세계 주류산업 동향"

# 가장 유사한 문서를 찾음
# k : 찾아올 갯수
docs = db.similarity_search(query, k=2)

print("[가장 유사한 문서]\n" + docs[0].page_content)
print("[그다음 유사한 문서]\n" + docs[1].page_content)

[가장 유사한 문서]
이 하루 종일 할 수 있는 종류의 일에 적합하다 라는 의견 제시human intern) ’
 ▸ 는 다양화된 추론을 위해 다중 을 활용하며 브라우저 코드 편집기와Manus LLM , DB, ,  
직접 상호작용하여 보고서 생성 또는 소프트웨어 디버깅(debugging)*을 수행
[그다음 유사한 문서]
해외동향 호AI.GOV 2025-1 7
주요 에이전트 서비스 AI Manus AI (Butterfly Effect, '25.3.)③ Ⅴ
 ▸ 는 중국의 스타트업 가 개발한 자율형 인공지능 에이전트로Manus Butterfly Effect , 
인간의 개입 없이 복잡한 작업을 독립적으로 수행할 수 있도록 설계


### Step 5 : Retriever 생성

In [19]:
# 5. Retriever
retriever = db.as_retriever()

### Step 6 : 프롬프트 생성

In [20]:
# 6. Prompt
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문에 답변하는 작업을 수행하는 어시스턴트입니다.
다음에 제공된 문맥 정보를 바탕으로 질문에 답하세요.
정답을 모를 경우, 모른다고만 말하세요.
답변은 반드시 한국어로 작성하세요.

#문맥:
{context}

#질문:
{question}

#답변:"""
)

### Step 7 : LLM 객체 생성

In [21]:
# 7. Model
from langchain.chat_models import init_chat_model

# 문맥기반으로 대답을 해야하기 때문에 창의성 0으로 설정
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.0)

### Step 8 : 체인 생성

In [22]:
# 8. Chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(), # 다음 체인으로 값을 그대로 넘김
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [23]:
# 9. 테스트
query = "주요 해외 AI 에이전트 서비스 동향에 대해 알려줘"
response = chain.invoke(query)

print(response)

주요 해외 AI 에이전트 서비스 동향에 따르면, 2025년에는 다양한 에이전트 서비스가 출시되고 있으며, 특히 'Computer Use', 'Operator', 'Manus AI'와 같은 서비스가 주목받고 있습니다. 이 중 Manus AI는 중국의 스타트업이 개발한 자율형 인공지능 에이전트로, 인간의 개입 없이 복잡한 작업을 독립적으로 수행할 수 있도록 설계되었습니다. 이러한 에이전트들은 인공지능 산업에서 중요한 트렌드로 자리잡고 있습니다.


In [24]:
llm.invoke(query)

AIMessage(content='2023년 현재, 해외 AI 에이전트 서비스는 다양한 분야에서 빠르게 발전하고 있으며, 여러 가지 주요 동향이 나타나고 있습니다. 다음은 그 중 몇 가지 주요 동향입니다.\n\n1. **개인화 및 맞춤형 서비스**: AI 에이전트는 사용자 데이터를 분석하여 개인 맞춤형 경험을 제공하는 데 중점을 두고 있습니다. 예를 들어, 추천 시스템이나 개인화된 콘텐츠 제공이 더욱 정교해지고 있습니다.\n\n2. **자연어 처리(NLP) 기술의 발전**: GPT-3와 같은 대규모 언어 모델의 발전으로 인해 AI 에이전트는 자연어 이해 및 생성 능력이 크게 향상되었습니다. 이는 고객 서비스, 콘텐츠 생성, 번역 등 다양한 분야에서 활용되고 있습니다.\n\n3. **멀티모달 AI**: 텍스트, 이미지, 음성 등 다양한 형태의 데이터를 동시에 처리할 수 있는 멀티모달 AI 기술이 주목받고 있습니다. 이는 사용자와의 상호작용을 더욱 풍부하게 만들어 줍니다.\n\n4. **AI 윤리 및 규제**: AI 기술의 발전과 함께 윤리적 문제와 규제에 대한 논의도 활발해지고 있습니다. 데이터 프라이버시, 편향성 문제, 투명성 등이 주요 이슈로 떠오르고 있으며, 이에 대한 해결책을 모색하는 노력이 진행되고 있습니다.\n\n5. **AI 기반 자동화**: 비즈니스 프로세스 자동화(BPA)와 로봇 프로세스 자동화(RPA) 분야에서 AI 에이전트의 활용이 증가하고 있습니다. 이는 기업의 효율성을 높이고 비용을 절감하는 데 기여하고 있습니다.\n\n6. **AI 에이전트의 통합**: 다양한 플랫폼과 서비스에 AI 에이전트를 통합하는 경향이 증가하고 있습니다. 예를 들어, CRM 시스템, 헬프데스크 소프트웨어, 전자상거래 플랫폼 등에 AI 에이전트를 통합하여 사용자 경험을 개선하고 있습니다.\n\n7. **AI 에이전트의 대화형 인터페이스**: 챗봇과 음성 비서와 같은 대화형 AI 에이전트의 사용이 증가하고 있으며, 이는 고객 지원, 정보 검색, 예약 시스템 등 다양한